# Demo - Fake Momo

**Real Momo** 
- most valuable flowdock member of January 2016
- responsible for restarting token server and sending free-token notifications for token users
- cheering up (or pissing off) token users when they are waiting for a solve
- recommending expensive places for lunch for BNE team (dunno why)

**Third-party packages:**
- Flowdock api wrapper: [pyflowdock](https://github.com/Aeron/PyFlowdock) - Simple Flowdock APIs wrapper
- Chatbot: [chatterbot](https://github.com/gunthercox/ChatterBot)

Also use: google places api, xkcd, dilbert, cowsay + fortune

## Flowdock API Libraries

See [A list of libraries to access CA Flowdock in your favorite language](https://www.flowdock.com/api/libraries) - e.g. Clojure, Erland, Go, "Noon's favourite", JavaScript, Perl, PHP, Python, R, Ruby

**Python**
- [PyFlowdock](https://github.com/Aeron/PyFlowdock/) by Eugene “Aeron” Glybin — Simple CA Flowdock APIs wrapper with some useful helpers
- [python-flowdock](https://bitbucket.org/j00bar/python-flowdock/wiki/Home) by Joshua Ginsberg — A Python interface to the CA Flowdock API


## pyflowdock demo
- Simple Flowdock APIs wrapper with some useful helpers. 
- Only Push API (Team Inbox and Chat)
- Streaming API available at this moment.

In [13]:
!pip install pyflowdock

In [2]:
# Flow - Momo
FLOW_API_TOKEN = ''
FLOW_EMAIL = 'momo@organisation_xxx.flowdock.com'

### Post to Flowdock Chat or Flowdock Team Inbox

In [5]:
from flowdock import Chat, TeamInbox

BOT_NAME = 'FakeMomo'


def post_to_chat(flow_api_token, msg, tag_list, sender_name=BOT_NAME):
    """
    Post to Flowdock Chat
    """
    chat = Chat(flow_api_token)
    
    # tag_list_str e.g. "#awesome,cool,@mike"
    chat.post(msg, sender_name, tag_list)


def post_to_team_inbox(flow_api_token, subject, msg, sender_email):
    """
    Post a Flowdock TeamInbox message
    """
    inbox = TeamInbox(flow_api_token)
    inbox.post('Source', sender_email, subject, '<p>{}</p>'.format(msg))


In [14]:
post_to_chat(FLOW_API_TOKEN, msg='Hello world', tag_list='#test,@dev01')

In [15]:
post_to_team_inbox(FLOW_API_TOKEN, subject="Build failed", msg="Just kidding!", sender_email=FLOW_EMAIL)

### Streaming specific flows

In [10]:
# Need 'User' details, only can access flows which user have access to (i.e. need to add this User to the flow)

# User Personal API Token (Momo's)
PERSONAL_API_TOKEN = ''


In [11]:
"""
Streaming specific flow in Flowdock

See also Flowdock message types: https://www.flowdock.com/api/message-types
"""
from flowdock import JSONStream
import json


def streaming():

    # Flow Name - please note that the flow name is the original name created
    # i.e. 'parameterized_name' in https://<username>:<password>:api.flowdock.com/flows
    flow_list = [ 'organisation_xxx/momo' ]

    # Streaming API supports two different content types, JSON stream and Event-Stream.
    stream = JSONStream(PERSONAL_API_TOKEN)

    try:
        gen = stream.fetch(flow_list, plain=True)
        
        for data_str in gen:
            data = json.loads(data_str)
            dump_data(data)

    except Exception as e:
        print e

        
def dump_data(data):
    print json.dumps(data, sort_keys=True, indent=4)


In [1]:
streaming()

NameError: name 'streaming' is not defined

See also [Flowdock message types and spectifications](https://www.flowdock.com/api/message-types)

### Examples of getting more from Flowdock API

In [23]:
"""
Prepare resources ACCESSIBLE by Momo (not all flows are accessible by Momo!)
"""
import requests


DEFAULT_CONTENT_TYPE = 'application/json'
FLOWS_API_URL = 'https://api.flowdock.com/flows'
FLOWS_ALL_API_URL = 'https://api.flowdock.com/flows/all'
USERS_API_URL = 'https://api.flowdock.com/users'


class MomoResources(object):
    def __init__(self, personal_api_token, logger):
        self.auth = (personal_api_token, '')
        self.logger = logger

        # [team_notifications, last_message_id, description, last_message_at,
        #  url, joined, id, parameterized_name, web_url, flow_admin, api_token,
        #  organization, access_mode, open, email, name]
        self.flows_json = MomoResources.api_request(FLOWS_API_URL, self.auth, self.logger)

        # [website, name, email, nick, avatar, id]
        self.users_json = MomoResources.api_request(USERS_API_URL, self.auth, self.logger)

    def api_token(self, id):
        """
        Return the Flow API Token for the given Flow ID
        """
        if self.flows_json:
            for f in self.flows_json:
                if f['id'].encode('utf-8') == id:
                    return f['api_token'].encode('utf-8')
        return None

    @property
    def flow_parameterized_names(self):
        """
        Return the parameterized_name of the list of flows accessible by Momo
        """
        return [f['parameterized_name'].encode('utf-8') for f in self.flows_json] \
            if self.flows_json is not None else []

    @property
    def organization_flow_parameterized_names(self):
        """
        Return the organization_name/parameterized_name of the list of flows accessible by Momo
        """
        return ['{}/{}'.format(
                f['organization']['parameterized_name'].encode('utf-8'),
                f['parameterized_name'].encode('utf-8')
            )
            for f in self.flows_json] \
            if self.flows_json is not None else []

    @staticmethod
    def api_request(api_url, auth, logger):
        """
        Process the actual Flowdock API GET requests
        """
        try:
            headers = {
                'content-type': DEFAULT_CONTENT_TYPE,
                'accept': DEFAULT_CONTENT_TYPE,
            }

            ret = requests.get(api_url, headers=headers, auth=auth)
            if ret.status_code == 200:
                return ret.json()

            logger.error('{}: status_code: {}'.format(api_url, ret.status_code))

        except Exception as e:
            logger.error('{} {}: {}'.format(api_url, type(e), e.message))
        return None



## Chatbot Libraries (python)

- [ChatterBot](https://pypi.python.org/pypi/ChatterBot/0.4.0)
    


## chatterbot demo

- [ChatterBot](https://pypi.python.org/pypi/ChatterBot/0.4.0)
- Each time a user enters a statement, the library saves the text that they entered and the text that the statement was in response to.
- Different basic [adapters/logic](https://github.com/gunthercox/ChatterBot/tree/master/chatterbot/adapters/logic)

In [25]:
!pip install chatterbot==0.3.5
!pip install python-Levenshtein==0.12.0

In [ ]:
from chatterbot import ChatBot

# Initialise with specific adapters
chatbot = ChatBot(
    "FakeMomo",
    logic_adapter="chatterbot.adapters.logic.closest_match.ClosestMatchAdapter",
    storage_adapter="chatterbot.adapters.storage.JsonDatabaseAdapter",
    io_adapter="chatterbot.adapters.io.TerminalAdapter",
    database='C:\Users\dev01\Desktop\database.db'
)

# Train based on the english corpus
chatbot.train("chatterbot.corpus.english")

In [12]:
# Get a response to an input statement
chatbot.get_response("Hello, how are you today?")

Hi


u'Hi'

## chatterbot + pyflowdock demo

In [ ]:
def process_data(data, chatbot):
    # only process original messages from Chat
    if 'app' not in data or data['app'] != 'chat' \
        or 'event' not in data or data['event'] != 'message':
        return

    # From myself
    if data['user'] == "0" \
        or data['content'] is None:
        return

    data_content = data['content'].encode('utf-8').strip()
    print 'Received: {}'.format(data_content)
    
    msg = chatbot.get_response(data_content)
    post_to_chat(FLOW_API_TOKEN, msg=msg, tag_list='#test,@dev01')


def streaming(chatbot):
    # Flow Name - please note that the flow name is the original name created
    # i.e. 'parameterized_name' in https://<username>:<password>:api.flowdock.com/flows
    flow_list = [ 'organisation_xxx/momo' ]
    stream = JSONStream(PERSONAL_API_TOKEN)

    try:
        gen = stream.fetch(flow_list, plain=True)
        
        for data_str in gen:
            data = json.loads(data_str)
            process_data(data, chatbot)

    except Exception as e:
        print e

In [ ]:
streaming(chatbot)

## Google Places API

In [48]:
"""
Google Places
"""
import json
import random
import urllib

DEFAULT_RADIUS_METRES = 1000
DEFAULT_TYPE = 'restaurant'


def google_places(lat_lng, key, type_str=DEFAULT_TYPE, radius=DEFAULT_RADIUS_METRES):

    try:
        # find possible options
        search_url = ('https://maps.googleapis.com/maps/api/place/nearbysearch/json'
                      '?location=%s'
                      '&radius=%s'
                      '&type=%s'
                      '&opennow'
                      '&key=%s') % (lat_lng, radius, type_str, key)

        response = urllib.urlopen(search_url)
        json_raw = response.read()
        json_data = json.loads(json_raw)
        
        if json_data['status'] == 'OK':
            results = json_data['results']

            if len(results) > 0:
                # just pick one
                random.seed()
                ret = results[random.randint(1, len(results))]

                link = 'https://www.google.com/maps/place/{}/@{},17z/'.format(
                    ret['name'].replace(' ','+'), lat_lng
                )

                msg = 'How about [{}, {}]({})?'.format(ret['name'], ret['vicinity'], link)
                return msg

        return 'No interesting place there! How about go somewhere else?'

    except Exception as e:
        print e

    return 'Not a good day to dine out! How about order pizzas over the phone?'


In [40]:
# TODO specify the api key here
GOOGLE_API_KEY = ''
WINDSOR_LAT_LNG = '-37.856159,144.9922'

google_places(lat_lng=WINDSOR_LAT_LNG, key=GOOGLE_API_KEY, type_str=DEFAULT_TYPE, radius=DEFAULT_RADIUS_METRES)


'How about [Borsch Vodka & Tears, 173 Chapel Street, Windsor](https://www.google.com/maps/place/Borsch+Vodka+&+Tears/@-37.856159,144.9922,17z/)?'

## xkcd

In [41]:
!pip install xkcd==2.3.1

In [45]:
import xkcd

# Retrieve url of a random xkcd
comic = xkcd.getRandomComic()
msg = '{}:\n{}\n{}'.format(
    comic.getAsciiTitle(),
    comic.getAsciiAltText(),
    comic.getAsciiImageLink()
)

post_to_chat(FLOW_API_TOKEN, msg, 'xkcd')


## dilbert

In [46]:
!pip install EasyProcess==0.2.2

In [47]:
import random
import re
import urllib

# Retrieve url of a random dilbert

year = random.choice(['2013', '2014', '2015'])
month = random.choice(range(1, 13))
day = random.choice(range(1, 29))

url_to_dilbert_page = 'http://www.dilbert.com/%s-%s-%s/' % (year, month, day)
page_contents = urllib.urlopen(url_to_dilbert_page).read()
msg = re.search('<meta name="twitter:image" content="(.*)">', page_contents).group(1) + '.png'

post_to_chat(FLOW_API_TOKEN, msg, 'dilbert')
